# Graphische Auswertung der Photovolaik-Anlage von 2018 bis 2020

<h1>Inhaltsverzeichnis<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Anzeige-Monatsmessungen" data-toc-modified-id="Anzeige-Monatsmessungen-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Anzeige Monatsmessungen</a></span></li><li><span><a href="#Anzeige-Tagesverlauf-(-im-15-Min-Takt)" data-toc-modified-id="Anzeige-Tagesverlauf-(-im-15-Min-Takt)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Anzeige Tagesverlauf ( im 15 Min Takt)</a></span></li><li><span><a href="#Anzeige-Durchschnittsdaten-pro-Wochentag" data-toc-modified-id="Anzeige-Durchschnittsdaten-pro-Wochentag-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Anzeige Durchschnittsdaten pro Wochentag</a></span></li><li><span><a href="#Tagesverlauf-mit-konfigurierbarem-Wochentag" data-toc-modified-id="Tagesverlauf-mit-konfigurierbarem-Wochentag-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Tagesverlauf mit konfigurierbarem Wochentag</a></span></li><li><span><a href="#Tagesvergleich-2018---2020" data-toc-modified-id="Tagesvergleich-2018---2020-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Tagesvergleich 2018 - 2020</a></span></li></ul></div>

In [1]:
import numpy as np 
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets


import plotly.express as px


In [2]:
DAILY_FILE = "https://raw.githubusercontent.com/enomis123/SolData/master/SolarEdgeDaily.csv"
HOURLY_FILE = "https://raw.githubusercontent.com/enomis123/SolData/master/SolarEdgeHourly.csv"
QUARTERY_FILE = "https://raw.githubusercontent.com/enomis123/SolData/master/SolarEdgeQuarterly.csv"

In [3]:
#Configuration ploty thema
mytheme = widgets.Dropdown(
    options= ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none'],
    value='seaborn',
    description='Darstellung (Theme):',
)

#container = widgets.HBox(children=[mytheme])
#widgets.VBox([container])


***

## Anzeige Monatsmessungen 

In [4]:
A1_data = pd.read_csv (DAILY_FILE, parse_dates = [0]) 
#data.dropna (inplace = True)
A1_data['Year'] = A1_data.Date.dt.year
A1_data['Month'] = A1_data.Date.dt.month
#data.sample(3)

In [5]:
A1_year = widgets.Dropdown(
    options=list(A1_data['Year'].unique()),
    value=2018,
    description='Jahr:',
)

A1_month = widgets.Dropdown(
    options=list(sorted (A1_data['Month'].unique())),
    value=2,
    description='Monat:',
)

A1_chart = widgets.Dropdown(
    options= ['barchart','linechart','scatterplot'],
    value='barchart',
    description='Chart:',
)

#Inital x values für Jan 2018 
xinit = A1_data[(A1_data['Date'].dt.year == 2018) & (A1_data['Date'].dt.month == 2)] 
xinit = xinit[['Date','Production','Consumption','SelfConsumption','FeedIn','Purchased']]

# Assign an empty figure widget with two traces
A1_trace1 = go.Bar(x = xinit.Date, y=xinit.Production, opacity=0.75, name='Stromerzeugung')
A1_trace2 = go.Bar(x = xinit.Date, y=xinit.Consumption, opacity=0.75, name='Stromverbrauch',visible='legendonly')
A1_trace3 = go.Bar(x = xinit.Date, y=xinit.SelfConsumption, opacity=0.75, name='Eigenverbrauch',visible='legendonly')
A1_trace4 = go.Bar(x = xinit.Date, y=xinit.FeedIn, opacity=0.75, name='Einspeisung',visible='legendonly')
A1_trace5 = go.Bar(x = xinit.Date, y=xinit.Purchased, opacity=0.75, name='Zukauf',visible='legendonly')
   
A1_g = go.FigureWidget(data=[A1_trace1, A1_trace2, A1_trace3, A1_trace4, A1_trace5],
                    layout=go.Layout(
                        barmode='group'
                    ))



In [6]:
def validate():
    if A1_year.value in A1_data['Year'].unique() and A1_month.value in A1_data['Month'].unique():
        return True
    else:
        return False


def response(change):
    #if validate():
    filter_list = [i and j for i, j in zip(A1_data['Month'] == A1_month.value, A1_data['Year'] == A1_year.value )]
    temp_df = A1_data[filter_list]
    x1 = temp_df[['Date','Production','Consumption','SelfConsumption','FeedIn','Purchased']]
    with A1_g.batch_update():
        A1_g.data[0].x = x1.Date
        A1_g.data[0].y = x1.Production
        A1_g.data[1].x = x1.Date
        A1_g.data[1].y = x1.Consumption
        A1_g.data[2].x = x1.Date
        A1_g.data[2].y = x1.SelfConsumption
        A1_g.data[3].x = x1.Date
        A1_g.data[3].y = x1.FeedIn
        A1_g.data[4].x = x1.Date
        A1_g.data[4].y = x1.Purchased
    
  
A1_year.observe(response, names="value")    
A1_month.observe(response, names="value")

In [7]:


A1_g.layout.title =""

container = widgets.HBox(children=[A1_year, A1_month])


A1_g.layout.xaxis.title = 'Datum'
A1_g.layout.yaxis.title = 'Menge in Wh'   

A1_g.update_layout(legend_title_text='Messwerte für', template ="seaborn")
A1_g.update_layout(template=mytheme.value) 
widgets.VBox([container, A1_g])

#TODO - wie bekomme ich den Titel nach oben ? 


## Anzeige Tagesverlauf ( im 15 Min Takt)

In [ ]:
#Einlesen der Werte
qData = pd.read_csv (QUARTERY_FILE, parse_dates = [0]) 
#data.dropna (inplace = True)
qData['Year'] = qData.Date.dt.year
qData['Month'] = qData.Date.dt.month
qData['Day'] = qData.Date.dt.day
#qData.sample(3)

In [ ]:
#Erstellen der Widgets 
year = widgets.Dropdown(
    options=list(qData['Year'].unique()),
    value=2018,
    description='Jahr:',
)

month = widgets.Dropdown(
    options=list(sorted (qData['Month'].unique())),
    value=2,
    description='Monat:',
)

day = widgets.Dropdown(
    options=list(sorted (qData['Day'].unique())),
    value=21,
    description='Tag:',
)

#Inital x values für Startdatum 21.2.2018
xinit = qData[(qData['Date'].dt.year == 2018) & (qData['Date'].dt.month == 2)& (qData['Date'].dt.day == 21) ] 
xinit = xinit[['Date','Production','Consumption','SelfConsumption','FeedIn','Purchased']]

# Assign an empty figure widget with two traces
trace1 = go.Bar(x = xinit.Date, y=xinit.Production, opacity=0.75, name='Stromerzeugung')
trace2 = go.Bar(x = xinit.Date, y=xinit.Consumption, opacity=0.75, name='Stromverbrauch',visible='legendonly')
trace3 = go.Bar(x = xinit.Date, y=xinit.SelfConsumption, opacity=0.75, name='Eigenverbrauch',visible='legendonly')
trace4 = go.Bar(x = xinit.Date, y=xinit.FeedIn, opacity=0.75, name='Einspeisung',visible='legendonly')
trace5 = go.Bar(x = xinit.Date, y=xinit.Purchased, opacity=0.75, name='Zukauf',visible='legendonly')
   
g = go.FigureWidget(data=[trace1, trace2, trace3, trace4, trace5],
                    layout=go.Layout(
                        barmode='group'
                    ))

In [ ]:
# Widgethandler
def response(change):
    filter_list = [i and j and k for i, j,k in zip(qData['Month'] == month.value, qData['Year'] == year.value, qData['Day'] == day.value, )]
    temp_df = qData[filter_list]
    x1 = temp_df[['Date','Production','Consumption','SelfConsumption','FeedIn','Purchased']]
    with g.batch_update():
        g.data[0].x = x1.Date
        g.data[0].y = x1.Production
        g.data[1].x = x1.Date
        g.data[1].y = x1.Consumption
        g.data[2].x = x1.Date
        g.data[2].y = x1.SelfConsumption
        g.data[3].x = x1.Date
        g.data[3].y = x1.FeedIn
        g.data[4].x = x1.Date
        g.data[4].y = x1.Purchased
        g.update_layout(title_text='Messwerte ' + x1.Date.head(1).dt.strftime('%d-%m-%Y').tolist()[0]
                        + " (" +x1.Date.head(1).dt.strftime('%A').tolist()[0] +")")
  
year.observe(response, names="value")    
month.observe(response, names="value")
day.observe(response, names="value")

In [ ]:
#Anzeige 

g.layout.title ="Messwerte 21-02-2018 (Wednesday)"

container = widgets.HBox(children=[year, month, day])

g.layout.xaxis.title = 'Uhrzeit'
g.layout.yaxis.title = 'Menge in Wh'   

g.update_layout(legend_title_text='Messwerte für')

g.update_layout(template=mytheme.value) 
widgets.VBox([container, g])


# TODO 3 SUBPLOTS fpr 2018, 2019, 2020 zum Vergleich , die alle durch die Widgets gesteuert werden ? 
# Durchschnittskurve einblenden ? 

# Lohnt sich dre Kauf einer Batterie: Purchased - FeedIn 

# 2 ) Wann zu welches Tageszeit wird der meiste Strom produziert 
# 1 ) An welchem Wochentag/ Tageszeit  wird der meiste Strom verbraucht 


##  Anzeige Durchschnittsdaten pro Wochentag 

In [ ]:
data = pd.read_csv (DAILY_FILE, parse_dates = [0]) 
#data.dropna (inplace = True)
data['Year'] = data.Date.dt.year
data['Month'] = data.Date.dt.month
data['WeekDay'] = data.Date.dt.weekday
#data['WeekDayName'] = data.Date.dt.strftime("%A")
#data.sample(3)

In [ ]:
df = data.groupby (['Year', 'WeekDay']).mean()/1000
df = df.round(2)

In [ ]:
def set_trace (type, v):
    wdays = ['Mo','Tues','Wed','Thurs','Fri','Sat','Sun']
    fig.add_trace(go.Bar(x=wdays, y= df[type].loc[2018],name="2018", visible=v))
    fig.add_trace(go.Bar(x=wdays, y= df[type].loc[2019],name="2019", visible=v))
    fig.add_trace(go.Bar(x=wdays, y= df[type].loc[2020],name="2020", visible=v))

In [ ]:
# Initialize figure

fig = go.Figure()

set_trace ('Consumption', True)
set_trace ('Production', False)
set_trace ('SelfConsumption', False)
set_trace ('FeedIn', False)
set_trace ('Purchased', False)

fig.update_layout(
    updatemenus=[

        dict(
            active=0,
            direction="down",
            type = "buttons",
            showactive =True,
            buttons=list([
                dict(label="Consumption",
                     method="update",
                     args=[{"visible": [True, True, True, False,False,False,False,False,False,False,False,False,False,False,False]},
                           {
                               #"title": "Consumption",
                           }]),
                dict(label="Production",
                     method="update",
                     args=[{"visible": [False, False, False, True,True,True,False,False,False,False,False,False,False,False,False]},
                           {
                               #"title": "Production" 
                           }]),
                dict(label="SelfConsumption",
                     method="update",
                     args=[{"visible": [False, False, False, False,False,False,True,True,True,False,False,False,False,False,False]},
                           {
                               #"title": "SelfConsumption"
                           }]),
                dict(label="FeedIn",
                     method="update",
                     args=[{"visible": [False, False, False, False,False,False,False,False,False,True,True,True,False,False,False]},
                           {
                               #"title": "FeedIn"
                           }]),
                dict(label="Purchased",
                     method="update",
                     args=[{"visible": [False, False, False, False,False,False,False,False,False,False,False,False,True,True,True]},
                           {
                               #"title": "Purchased" 
                           }]),
            ]),
        )
    ])


# Set title
fig.update_layout(title_text="Vergleich Durchschnittswerte pro Wochentag")
#fig.update_layout (annotations=[
#                     dict(text="Lines", x=0, xref="paper", y= 1.1,yref="paper",
#                             showarrow=False)
#    ])


fig.update_layout(template=mytheme.value) 
fig.show()

## Tagesverlauf mit konfigurierbarem Wochentag

In [ ]:
qData = pd.read_csv ("/Volumes/MyUsers/simone/CAS_StatDat/arbeit/SolaredgeQuarterly.csv", parse_dates = [0]) 
qData['Year'] = qData.Date.dt.year
#qData['Month'] = qData.Date.dt.month
qData['WeekDay'] = qData.Date.dt.weekday
qData['Time'] = qData.Date.dt.strftime('%H:%M')

In [ ]:
df = qData.groupby (['Year', 'WeekDay','Time']).mean()
df = df.round(2)
df.reset_index(inplace = True)

In [ ]:
type = widgets.Dropdown(
    options= ['Consumption','Production','SelfConsumption','FeedIn','Purchased'],
    value='Consumption',
    description='Type:',
)

weekday = widgets.Dropdown(
    options=['Montag','Dienstag','Mittwoch','Donnerstag','Freitag','Samstag','Sonntag'],
    value='Montag',
    description='Wochentag:',
)

# Assign an empty figure widget with two traces
trace1 = go.Scatter(x = df[df.Year==2018].Time.unique(), y=df[df.Year==2018].Consumption, opacity=0.75, name='2018')
trace2 = go.Scatter(x = df[df.Year==2019].Time.unique(), y=df[df.Year==2019].Consumption, opacity=0.75, name='2019')
trace3 = go.Scatter(x = df[df.Year==2020].Time.unique(), y=df[df.Year==2020].Consumption, opacity=0.75, name='2020')
   
g = go.FigureWidget(data=[trace1, trace2, trace3],
                    layout=go.Layout(
                        barmode='group'
                    ))

In [ ]:
wdict = {
  "Montag": 0,
  "Dienstag": 1,
  "Mittwoch": 2,
  "Donnerstag": 3,
  "Freitag": 4,
  "Samstag": 5,
 "Sonntag": 6,   
}


def response(change):
    temp_df = df[df.WeekDay==wdict[weekday.value]]
    x1 =temp_df[['Year','Time',type.value]]
    with g.batch_update():
        g.data[0].x = x1.Time
        g.data[0].y = x1 [x1.Year ==2018][type.value]
        #g.data[1].x = x1.Time
        g.data[1].y = x1 [x1.Year ==2019][type.value]
        #g.data[2].x = x1.Time
        g.data[2].y = x1 [x1.Year ==2020][type.value]
    
  
type.observe(response, names="value")    
weekday.observe(response, names="value")

In [ ]:
#g.layout.title ="Messwert XXX"

container = widgets.HBox(children=[type, weekday])

g.layout.xaxis.title = 'Uhrzeit'
g.layout.yaxis.title = 'Wh'   
g.layout.yaxis.range = [0,1700]
g.update_layout(legend_title_text='Jahr:')
g.update_layout(template=mytheme.value) 
widgets.VBox([container, g])

## Tagesvergleich 2018 - 2020

In [ ]:
dData = pd.read_csv (DAILY_FILE, parse_dates = [0]) 
dData['Year'] = dData.Date.dt.year
dData['Month'] = dData.Date.dt.month
dData['Day'] = dData.Date.dt.day
#dData.sample(3)

In [ ]:
type = widgets.Dropdown(
    options= ['Consumption','Production','SelfConsumption','FeedIn','Purchased'],
    value='Consumption',
    description='Type:',
)

month = widgets.Dropdown(
    options=['Januar','Februar','Maerz','April','Mai','Juni','July','August','September','Oktober','November','Dezember'],
    value='Februar',
    description='Monat:',
)


mdict = {
   "Januar": 1, 
    "Februar": 2, 
    "Maerz": 3, 
    "April": 4, 
    "Mai": 5, 
    "Juni": 6, 
    "July": 7, 
    "August": 8, 
    "September": 9, 
    "Oktober": 10, 
    "November": 11, 
    "Dezember": 12
}


d2018 = dData[ (dData.Year==2018) & (dData.Month==2)]
d2019 = dData[ (dData.Year==2019) & (dData.Month==2)]
d2020 = dData[ (dData.Year==2020) & (dData.Month==2)]


# Assign an empty figure widget with two traces
trace1 = go.Bar(x = d2018.Day, y=d2018.Consumption, opacity=0.75, name='2018')
trace2 = go.Bar(x = d2019.Day, y=d2019.Consumption, opacity=0.75, name='2019')
trace3 = go.Bar(x = d2020.Day, y=d2020.Consumption, opacity=0.75, name='2020')

   
g = go.FigureWidget(data=[trace1, trace2, trace3],
                    layout=go.Layout(
                        barmode='group'
                    ))

In [ ]:
def response(change):
    temp_df = dData[dData.Month==mdict[month.value]]
    x1 =temp_df[['Year','Day',type.value]]
    with g.batch_update():
        g.data[0].x = x1.Day
        g.data[0].y = x1 [x1.Year ==2018][type.value]
        #g.data[1].x = x1.Time
        g.data[1].y = x1 [x1.Year ==2019][type.value]
        #g.data[2].x = x1.Time
        g.data[2].y = x1 [x1.Year ==2020][type.value]
    
  
type.observe(response, names="value")    
month.observe(response, names="value")

In [ ]:
container = widgets.HBox(children=[type, month])

g.layout.xaxis.title = 'Tag'
g.layout.yaxis.title = 'Wh'   
#g.layout.yaxis.range = [0,1700]
g.update_layout(legend_title_text='Jahr:')


g.update_layout(template=mytheme.value) 


#px.colors.sequential.Plasma_r
#color_continuous_scale=px.colors.sequential.Viridis
widgets.VBox([container, g])